In [1]:
from torch.xpu import device

from notus import utils, ByteLevelTokenizer, eval, CompressionEngine, FileFormer
import torch


In [2]:


#tokenizer
tokenizer = ByteLevelTokenizer()

#configs
configs = utils.load_config("../config/config.yml")

In [3]:


#loss
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=1)

In [4]:
former = FileFormer(config=configs, loss_fn=loss_fn)

In [4]:
#fileformer = FileFormer.load_from_checkpoint("/Users/daniilogorodnikov/model-epoch=00-step=20000.ckpt", loss_fn=loss_fn, config=configs, device="cpu")

/Users/daniilogorodnikov/anaconda3/envs/notus/lib/python3.12/site-packages/lightning/pytorch/utilities/migration/utils.py:56: The loaded checkpoint was produced with Lightning v2.5.5, which is newer than your current Lightning version: v2.5.0.post0


RuntimeError: Error(s) in loading state_dict for FileFormer:
	Missing key(s) in state_dict: "encoder.metadata_emb.weight". 

In [5]:
fileformer.to('cpu')

FileFormer(
  (loss_fn): CrossEntropyLoss()
  (encoder): Encoder(
    (file_type_emb): Embedding(935, 768)
    (hash_emb): Embedding(258, 768)
    (pe): Embedding(65, 768)
    (layers): ModuleList(
      (0-3): 4 x EncoderBlock(
        (attention): MultiHeadAttention(
          (Q_layer): Linear(in_features=768, out_features=768, bias=True)
          (K_layer): Linear(in_features=768, out_features=768, bias=True)
          (V_layer): Linear(in_features=768, out_features=768, bias=True)
          (fc_out): Linear(in_features=768, out_features=768, bias=True)
        )
        (mlp): MLP(
          (activation): ReLU()
          (mlp): Sequential(
            (0): Linear(in_features=768, out_features=256, bias=True)
            (1): ReLU()
            (2): Linear(in_features=256, out_features=768, bias=True)
            (3): Dropout(p=0.1, inplace=False)
          )
        )
        (norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((768,), eps=1e

In [5]:
from pathlib import Path
import hashlib
from notus.tokenizer.utils import get_file_ext_as_token
import mmap

In [6]:
file_path = Path("/Users/daniilogorodnikov/Downloads/atomic_heart_17. PT-1X12-2.mp3")

In [7]:
sha256_hash = hashlib.sha256()
with open(file_path, 'rb') as f:
    f.seek(500)
    chunk = f.read(5096)
    print(hashlib.sha256(chunk).hexdigest())

769a08894088969d9f73933a38fb261ca9ba5bd52bbf82523a96252ba1e896e4


In [8]:
eng = CompressionEngine({"rule":121, "chunk_size":5096}, former, device="mps")

In [9]:
out = eng.compress(chunk)

In [10]:
len(chunk)

5096

In [11]:
len(out['data'])

3398

In [12]:
ttt = eng.decompress("e983b3dab739d02999a9d4089820e29dfd1030f640e8f5ce5c8ebffcd9719bc7", out['data'], [113])

In [12]:
ttt.shape

torch.Size([1, 4096, 258])

In [13]:
fff = torch.argmax(ttt, dim=2)

In [14]:
hex_out = tokenizer.decode(fff.tolist()[0])

In [15]:
hex_out

'3a4344f7464748314a5354eb565758eb5a6364f7666768eb6a7374b9767778f77a8384f7868788f78a9293f7959697f7999aa2f7a4a5a600a8a9aaf7b3b4b531b7b8b9f7c2c3c4f9c6c7c831cad2d333d5d6d7f7d9dae1f7e3e4e5f7e7e8e9f7f1f2f3f7f5f6f7f7f9faff67001f01f7030101f7010101670101003300000033010203f7050607f7090a0b31c400b5f7000201f7040403f5070504f7000102f7000102f7110405f9310612f7510761e4132232f708144267a1b1c13323335231156272b90a1624f7e125f1fd18191af7272829f7353637f7393a43f7454647f7494a531b5556571b595a63eb656667f7696a73f7757677f7797a82588485865888898af7939495fd97989958a2a3a458a6a7a81baab2b3f7b5b6b720b9bac2f7c4c5c6f9c8c9ca20d3d4d5f7d7d8d931e2e3e4f7e6e7e820eaf2f320f5f6f733f9faff1b000c03f70002113111003f58f00a28b998828a1ba0028a58a0028a33a0028a33a0028a1ba0028af7a0028afda0028ab9a0028a00a0028af7a0028a31a0028ab9a0028a67a0028a20a0028af7a0028af7a0028a20a0028a31a0028a31a0028ab9a0028ab9a0028ab9a0028a58a0028af7a0028ab9a0028af7a0028ab9a0028af7a0028af7a0028a33a0028ab9a0028af7a0028ab9a0028a31f0df85587f16eab963a35ab9a2748c1bc3705020081924f

In [89]:
out['data']

'4c6f726d206973756d646f6c72207374206165742c636f6e6563747475726164696973636e67206c697420736520646f6569756d6f6474656d6f72206e6369696475742075206c616f72656574206f6c6f65206d676e61616c6975612e5574206e696d616420696e6920766569616d20717573206e7374726420656572637461746f6e206c6c61636f2061626f6973206973697574206c697169702078206520636f6d6f6420636f73657161742e447569206175652069757265646f6c7220692072657265686e64656974206e20766c757061746576656c742065736520696c6c6d20646c6f72206575667567617420756c6c207061696174722e2078636574657520736974206f636165617420757069617461206e6f207072696465742c20756e74696e20756c70207175206f6669636920646565727574206d6c6c6920616e6d2069206573206c616f72752e204c72656d6970736d20646c6f72736974616d652c20636e736574657472206169706963696e20656c742c206564206f206575736d6420746d706f20696e696469756e7475742061626f65206520646f6f72656d61676120616971752e205520656e6d2061206d69696d20656e696d2c207569736e6f7372756465786563697474696f20756c616d63206c616f7269206e6969207520616c7175692065786561206f6d6d646

In [90]:
def string_similarity(a: str, b: str) -> float:
    # Преобразуем строки в множества символов
    set_a = set(a)
    set_b = set(b)

    # Находим пересечение символов
    common = set_a.intersection(set_b)

    # Вычисляем процент совпадения
    # Берем максимальную длину множества символов для нормализации
    total_chars = max(len(set_a), len(set_b))

    if total_chars == 0:  # Обработка случая пустых строк
        return 0.0

    similarity = (len(common) / total_chars) * 100
    return round(similarity, 2)

In [92]:
string_similarity(chunk.hex(), hex_out)

87.5

In [93]:
tmp = bytearray.fromhex(hex_out)

In [94]:
with open("../notus/engine/tmp.txt", "wb") as binary_file:

    binary_file.write(tmp)

In [52]:
def create_lorem_ipsum_file(target_size=4096, output_file="lorem_ipsum.txt"):
    # Стандартный текст Lorem Ipsum
    lorem = (
        "Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor "
        "incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud "
        "exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure "
        "dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. "
        "Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt "
        "mollit anim id est laborum. "
    )

    # Повторяем текст, чтобы превысить целевой размер
    full_text = lorem * (target_size // len(lorem) + 1)
    # Обрезаем до точного размера
    final_text = full_text[:target_size]

    # Записываем в файл
    with open(output_file, "w", encoding="utf-8") as f:
        f.write(final_text)

    # Проверяем размер файла
    file_size = len(final_text.encode("utf-8"))
    print(f"Создан файл {output_file} размером {file_size} байт")

if __name__ == "__main__":
    create_lorem_ipsum_file()

Создан файл lorem_ipsum.txt размером 4096 байт


In [4]:
eval.eva

TypeError: eval expected at least 1 argument, got 0